# Account B - Replication Scenario
#### Read replicated feature group(s) residing in this account 

#### Prerequisites

In [ ]:
#!pip install awswrangler

#### Imports 

In [ ]:
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker import get_execution_role
from sagemaker.session import Session
import awswrangler as wr
import pandas as pd
import sagemaker
import logging
import boto3
import time
import s3fs

#### Setup Logger

In [ ]:
logger = logging.getLogger('sagemaker')
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

In [ ]:
logger.info(f'[Using SageMaker version: {sagemaker.__version__}]')

#### Essentials 
* Create SageMaker & Feature Store Runtime Clients
* Create a Feature Store Session encapsulating the above clients
* Ensure the Execution Role you use for this notebook has all the policies attached to it as per the instructions manual. If not, please make sure to attach them to the role before proceeding.

In [ ]:
region = boto3.Session().region_name
boto_session = boto3.Session(region_name=region)
s3 = boto_session.resource('s3', region_name=region)
role = get_execution_role()

s3_client = boto3.client('s3', region_name=region)
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

In [ ]:
role

https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_featurestore.html <br>
API Documentation: https://sagemaker.readthedocs.io/en/stable/api/prep_data/feature_store.html

In [ ]:
feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

#### Offline Store Data Path

In [ ]:
bucket = 'sagemaker-fs-replication-account-b'

In [ ]:
prefix = 'project-1'
offline_feature_store_s3_uri = f's3://{bucket}/{prefix}'
offline_feature_store_s3_uri

#### KMS Custom Key for Encryption 

In [ ]:
sagemaker_client.list_feature_groups()

#### Validate if we can access the replicated objects sitting inside Account B's (this account) Offline Store

In [ ]:
# account ID of account A since the replication creates a folder with the name 
# of the account where the replication rule was created = Account A
account_id = '<ACCOUNT-ID-A>' 

In [ ]:
feature_group_name = 'employees-account-a' # the replica of the feature group created in Account A
feature_group_s3_prefix = f'{prefix}/{account_id}/sagemaker/{region}/offline-store/{feature_group_name}/data'
feature_group_s3_prefix

In [ ]:
offline_store_contents = None
while offline_store_contents is None:
    objects = s3_client.list_objects(Bucket=bucket, Prefix=feature_group_s3_prefix)
    if 'Contents' in objects and len(objects['Contents']) > 1:
        logger.info('[Features are available in Offline Store!]')
        offline_store_contents = objects['Contents']
    else:
        logger.info('[Waiting for data in Offline Store ...]')
        time.sleep(60)

In [ ]:
offline_store_contents

#### Inspect the Parquet Files (Offline Store)

In [ ]:
s3_prefix = '/'.join(offline_store_contents[0]['Key'].split('/')[:-5])
s3_uri = f's3://{bucket}/{s3_prefix}'
s3_uri

In [ ]:
df = wr.s3.read_parquet(path=s3_uri)

In [ ]:
df